# Задание по теме «Коллаборативная фильтрация»
ПАКЕТ SURPRISE

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

## Решение

### Загрузим библиотеку Surprise 
библиотека для работы с рекомендательными системами


In [1]:
!pip install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-win_amd64.whl size=747019 sha256=fdc46fc0dab6ed36ed9f3e17c1aa3a8ae674e460e4f5214dac3af3d62e5fd78e
  Stored in directory: c:\users\s\appdata\local\pip\cache\wheels\20\91\57\2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise


In [16]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy # всевозможные метрики
from surprise import Reader # класс считывания данных
from surprise.model_selection import train_test_split
from surprise.model_selection.validation import cross_validate

import pandas as pd

### Загрузим данные

In [4]:
#User information (users.dat table, data without entry information)
unames = ['user_id','gender','age','occupation','zip']
users = pd.read_table('users.dat', sep='::', header=None, names=unames, engine='python')

#Rating information
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_table('ratings.dat', sep='::', header=None, names=rnames, engine='python')

# #Movie information
mnames = ['movie_id','title','genres']
movies = pd.read_table('movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='ISO-8859-1')

In [5]:
data=pd.merge(pd.merge(ratings,users),movies)

In [6]:
data.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [8]:
dataset = pd.DataFrame({
    'uid': data.user_id,
    'iid': data.movie_id,
    'rating': data.rating 
})

Определим диапозон значений рейтинга

In [9]:
ratings.rating.min()

1

In [10]:
ratings.rating.max()

5

In [12]:
reader = Reader(rating_scale=(1.0, 5.0))
dt = Dataset.load_from_df(dataset, reader) 

In [13]:
trainset, testset = train_test_split(dt, test_size=.15)

#### Модель unuser_based KNNWithMeans

In [14]:
algo_2 = KNNWithMeans(k=100, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [17]:
cross_validate(algo_2, dt, measures=[u'rmse'], cv=5, n_jobs=1)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.86214225, 0.86435008, 0.86203754, 0.86132085, 0.86566056]),
 'fit_time': (56.30263686180115,
  55.028175592422485,
  55.241262435913086,
  55.31533169746399,
  55.63123917579651),
 'test_time': (99.51333022117615,
  102.91266703605652,
  102.43425130844116,
  101.91723489761353,
  101.78680229187012)}

#### Модель user_based KNNWithMeans

In [18]:
algo = KNNWithMeans(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [19]:
cross_validate(algo, dt, measures=[u'rmse'], cv=5, n_jobs=1)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.89046235, 0.88978208, 0.89104355, 0.88969568, 0.88907212]),
 'fit_time': (116.53715443611145,
  116.31845211982727,
  117.00023651123047,
  116.19446873664856,
  116.10019254684448),
 'test_time': (164.2922785282135,
  166.77027702331543,
  164.04313731193542,
  173.6767716407776,
  168.82912278175354)}